# My solution in efficientnet

## step1.training efn-b0 in public train set and val in private test set  LB=0.1856
## steg2.add no landmark filter LB=0.2069
## step3.finetuen efn-b0 in private train set

# 1.Clean Test set
[original notebook](https://www.kaggle.com/rhtsingh/pytorch-landmark-or-non-landmark-identification/)

In [ ]:
import pandas as pd
from tqdm import tqdm
import cv2
import torch
from torch.autograd import Variable as V
import torchvision.models as models
from torchvision import transforms as trn
from torch.nn import functional as F
import os
import numpy as np
import cv2
from PIL import Image
import gc
gc.enable()
import matplotlib.pyplot as plt

import sys
sys.path.append('../input/landmark-20-clean/')
import wideresnet

### Load classes and I/O labels of Places365 Dataset

In [ ]:
def load_labels():
    # prepare all the labels
    # scene category relevant
    file_name_category = '../input/landmark-20-clean/categories_places365.txt'
    classes = list()
    with open(file_name_category) as class_file:
        for line in class_file:
            classes.append(line.strip().split(' ')[0][3:])
    classes = tuple(classes)

    # indoor and outdoor relevant
    file_name_IO = '../input/landmark-20-clean/IO_places365.txt'
    with open(file_name_IO) as f:
        lines = f.readlines()
        labels_IO = []
        for line in lines:
            items = line.rstrip().split()
            labels_IO.append(int(items[-1]) -1) # 0 is indoor, 1 is outdoor
    labels_IO = np.array(labels_IO)
    return classes, labels_IO

### Image Transformations

In [ ]:
def returnTF():
# load the image transformer
    tf = trn.Compose([
        trn.ToPILImage(),
        trn.Resize((224,224)),
        trn.ToTensor(),
        trn.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    return tf

### Load Pretrained Weights & Create Model

In [ ]:
def hook_feature(module, input, output):
    features_blobs.append(np.squeeze(output.data.cpu().numpy()))
    
def recursion_change_bn(module):
    if isinstance(module, torch.nn.BatchNorm2d):
        module.track_running_stats = 1
    else:
        for i, (name, module1) in enumerate(module._modules.items()):
            module1 = recursion_change_bn(module1)
    return module

def load_model():
    # this model has a last conv feature map as 14x14
    model_file = '../input/landmark-20-clean/wideresnet18_places365.pth.tar'

    model = wideresnet.resnet18(num_classes=365)
    checkpoint = torch.load(model_file, map_location=lambda storage, loc: storage)
    state_dict = {str.replace(k,'module.',''): v for k,v in checkpoint['state_dict'].items()}
    model.load_state_dict(state_dict)
    
    # hacky way to deal with the upgraded batchnorm2D and avgpool layers...
    for i, (name, module) in enumerate(model._modules.items()):
        module = recursion_change_bn(model)
    model.avgpool = torch.nn.AvgPool2d(kernel_size=14, stride=1, padding=0)

    model.eval()
    # hook the feature extractor
    features_names = ['layer4','avgpool'] # this is the last conv layer of the resnet
    for name in features_names:
        model._modules.get(name).register_forward_hook(hook_feature)
    return model

### Config

In [ ]:
classes, labels_IO = load_labels()
features_blobs = []
device = torch.device("cuda") if torch.cuda.is_available() else "cpu"
model = load_model()
model = model.to(device)

tf = returnTF()

params = list(model.parameters())
weight_softmax = params[-2].data.cpu().numpy()
weight_softmax[weight_softmax<0] = 0

### Prediction Loop

In [ ]:
test_clean = pd.read_csv('../input/landmark-recognition-2020/sample_submission.csv')
io_test = []
for i, img_id in tqdm(enumerate(test_clean.id), total=len(test_clean)):
    image_path = f"../input/landmark-recognition-2020/test/{img_id[0]}/{img_id[1]}/{img_id[2]}/{img_id}.jpg"
    img = cv2.imread(image_path, cv2.IMREAD_COLOR)
    input_img = V(tf(img).unsqueeze(0))
    logit = model.forward(input_img.to(device))
    h_x = F.softmax(logit, 1).data.squeeze()
    probs, idx = h_x.sort(0, True)
    probs = probs.cpu().numpy()
    idx = idx.cpu().numpy()

    io_image = np.mean(labels_IO[idx[:10]]) # vote for the indoor or outdoor
    if io_image < 0.5:
        io_test.append(0) 
    else:
        io_test.append(1) 
        
    del input_img
    del img
    del image_path
    del logit
    del probs
    del idx
    del io_image
    if i%1000 ==0:
        gc.collect()

In [ ]:
test_clean['io'] = io_test
# test_clean.to_csv('test_io.csv',index=False)
test_clean.head()

In [ ]:
# int(test_clean[test_clean.id=='00084cdf8f600d00'].io)
io_labels = np.array((test_clean.io))
print(io_labels.shape)

### show img landmark or no landmark

In [ ]:
def return_img(img_id):
    image_path = f"../input/landmark-recognition-2020/test/{img_id[0]}/{img_id[1]}/{img_id[2]}/{img_id}.jpg"
    img = np.array(Image.open(image_path).resize((224, 224), Image.LANCZOS))
    return img

In [ ]:
# Get 'landmark' images
n = 16
landmark_images =  test_clean[test_clean['io'] == 1]['id'][:n]

fig = plt.figure(figsize = (16, 16))
for i, img_id in enumerate(landmark_images):
    image = return_img(img_id)
    fig.add_subplot(4, 4, i+1)
    plt.title(img_id)
    plt.imshow(image)

In [ ]:
# Get non 'landmark' images
n = 16
landmark_images =  test_clean[test_clean['io'] == 0]['id'][:n]
# landmark_indexes = landmark_images[:n].index.values

# Plot image examples
fig = plt.figure(figsize = (16, 16))
for i, img_id in enumerate(landmark_images):
    image = return_img(img_id)
    fig.add_subplot(4, 4, i+1)
    plt.title(img_id)
    plt.imshow(image)

# 2.[PyTorch Inference Submission] EfficientNet Baseline From [original notebook](https://www.kaggle.com/rhtsingh/pytorch-training-inference-efficientnet-baseline)

*Note: I have exhausted my GPU for this week and so was unable to complete training. When training started I had only 1/30hr left.*


### Settup Dependencies

In [ ]:
# !pip install efficientnet_pytorch
# !pip install torch_optimizer

In [ ]:
!pip install ../input/efficientnet-pytorch/EfficientNet-PyTorch-master/ > /dev/null # no output

In [ ]:
import os
import gc
gc.enable()
import sys
import math
import json
import time
import random
from glob import glob
from datetime import datetime

import cv2
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import multiprocessing
from sklearn.preprocessing import LabelEncoder

import torch
import torchvision
from torch import Tensor
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.nn.parameter import Parameter
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler
from tqdm import tqdm

import efficientnet_pytorch

# import torch_optimizer as optim
import torch.optim as optim
import albumentations as A

import sklearn

import warnings
warnings.filterwarnings("ignore")

### Train Configuration

*Note: Lots of improvement can be done simply here. e.g.*

* MIN SAMPLES PER CLASS - This variable is a threshold for total number of images in a class. If has class has less than this count then it will be discarded from training set.
* BATCH SIZE            - The number of images in each training batch.
* EPOCHS                - Total number of epochs.

> ### Dataset

image size=(3, 224, 224)

In [ ]:
class ImageDataset(torch.utils.data.Dataset):
    def __init__(self, dataframe: pd.DataFrame, image_dir:str, mode: str):
        self.df = dataframe
        self.mode = mode
        self.image_dir = image_dir
        
        transforms_list = []
        if self.mode == 'train':
            # Increase image size from (64,64) to higher resolution,
            # Make sure to change in RandomResizedCrop as well.
            transforms_list = [
                transforms.Resize((224,224)),
                transforms.RandomHorizontalFlip(),
                transforms.RandomChoice([
                    transforms.RandomResizedCrop(224),
                    transforms.ColorJitter(0.2, 0.2, 0.2, 0.2),
                    transforms.RandomAffine(degrees=15, translate=(0.2, 0.2),
                                            scale=(0.8, 1.2), shear=15,
                                            resample=Image.BILINEAR)
                ]),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                      std=[0.229, 0.224, 0.225]),
            ]
        else:
            transforms_list.extend([
                # Keep this resize same as train
                transforms.Resize((224,224)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                      std=[0.229, 0.224, 0.225]),
            ])
        self.transforms = transforms.Compose(transforms_list)

    def __getitem__(self, index: int):
        image_id = self.df.iloc[index].id
        image_path = f"{self.image_dir}/{image_id[0]}/{image_id[1]}/{image_id[2]}/{image_id}.jpg"
        image = Image.open(image_path)
        image = self.transforms(image)

        if self.mode == 'test':
            return {'image':image}
        else:
            return {'image':image, 
                    'target':self.df.iloc[index].landmark_id}

    def __len__(self) -> int:
        return self.df.shape[0]

### Load Data

In [ ]:
def load_data(public_train, train, test, train_dir, test_dir):
    counts = public_train.landmark_id.value_counts()
    selected_classes = counts[counts >= MIN_SAMPLES_PER_CLASS].index
    num_classes = selected_classes.shape[0]
    print('classes with at least N samples:', num_classes)

#     public_train = public_train.loc[public_train.landmark_id.isin(selected_classes)]
    print('public train_df', public_train.shape)
    print('train_df', train.shape)
    print('test_df', test.shape)

    # filter non-existing test images
    exists = lambda img: os.path.exists(f'{test_dir}/{img[0]}/{img[1]}/{img[2]}/{img}.jpg')
    test = test.loc[test.id.apply(exists)]
    print('test_df after filtering', test.shape)

    label_encoder = LabelEncoder()
    label_encoder.fit(public_train.landmark_id.values)
    print('found classes', len(label_encoder.classes_))
    assert len(label_encoder.classes_) == num_classes
    
    print(list(public_train.landmark_id)[:10])
    print(list(train.landmark_id)[:10])

    train.landmark_id = label_encoder.transform(train.landmark_id)

    train_dataset = ImageDataset(train, train_dir, mode='train')
    test_dataset = ImageDataset(test, test_dir, mode='test')

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE,
                              shuffle=True, num_workers=4, drop_last=True)

    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=False, num_workers=NUM_WORKERS)

    return train_loader, test_loader, label_encoder, num_classes, train.shape[0]

### Model

*Note: Used efficientnet-b0. Experimenting with different archs can yield different results*

In [ ]:
def stop_weight_updata(model):
#     for layer in list(model.children()):
#         for i, p in enumerate(layer.parameters()):
#             print(i, p.requires_grad)
    print('_______________________________________________________')
    for layer in list(model.children())[:-1]:
        for i, p in enumerate(layer.parameters()):
            p.requires_grad = False
#     print('_______________________________________________________')
#     for layer in list(model.children()):
#         for i, p in enumerate(layer.parameters()):
#             print(i, p.requires_grad)
            

class EfficientNetEncoderHead(nn.Module):
    def __init__(self, depth, num_classes=81313):
        super(EfficientNetEncoderHead, self).__init__()
        self.depth = depth
        model_name = 'efficientnet-b' + str(self.depth)
#         self.base = efficientnet_pytorch.EfficientNet.from_pretrained(f'efficientnet-b{self.depth}')
        self.base = efficientnet_pytorch.EfficientNet.from_name(f'efficientnet-b{self.depth}')
#         self.base.load_state_dict(torch.load(efn_weights[model_name]))
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.output_filter = self.base._fc.in_features
        self.classifier = nn.Linear(self.output_filter, num_classes)
    def forward(self, x):
        x = self.base.extract_features(x)
        x = self.avg_pool(x).squeeze(-1).squeeze(-1)
        x = self.classifier(x)
        return x

In [ ]:
# Test model
# model = EfficientNetEncoderHead(depth=0, num_classes=81313)
# try:
#     model_path = '../input/efnb0-10-gap05917pth/efn-b0_10_GAP0.5917.pth'
#     model.load_state_dict(torch.load(model_path, map_location='cpu'))
#     print('Model found in {}'.format(model_path))
# except:
#     print('Random initialize model!')

# model.cuda()
# stop_weight_updata(model)

### Inference Function

In [ ]:
def inference(data_loader, model):
    model.eval()

    activation = nn.Softmax(dim=1)
    all_predicts, all_confs, all_targets = [], [], []

    with torch.no_grad():
        for i, data in enumerate(tqdm(data_loader, disable=IN_KERNEL)):
            if data_loader.dataset.mode != 'test':
                input_, target = data['image'], data['target']
            else:
                input_, target = data['image'], None

            output = model(input_.cuda())
            output = activation(output)

            confs, predicts = torch.topk(output, NUM_TOP_PREDICTS)
            all_confs.append(confs)
            all_predicts.append(predicts)

            if target is not None:
                all_targets.append(target)

    predicts = torch.cat(all_predicts)
    confs = torch.cat(all_confs)
    targets = torch.cat(all_targets) if len(all_targets) else None

    return predicts, confs, targets

## trainning loop

In [ ]:
class AverageMeter:
    ''' Computes and stores the average and current value '''
    def __init__(self) -> None:
        self.reset()

    def reset(self) -> None:
        self.val = 0.0
        self.avg = 0.0
        self.sum = 0.0
        self.count = 0

    def update(self, val: float, n: int = 1) -> None:
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def GAP(predicts: torch.Tensor, confs: torch.Tensor, targets: torch.Tensor) -> float:
    ''' Simplified GAP@1 metric: only one prediction per sample is supported '''
    assert len(predicts.shape) == 1
    assert len(confs.shape) == 1
    assert len(targets.shape) == 1
    assert predicts.shape == confs.shape and confs.shape == targets.shape

    _, indices = torch.sort(confs, descending=True)

    confs = confs.cpu().numpy()
    predicts = predicts[indices].cpu().numpy()
    targets = targets[indices].cpu().numpy()

    res, true_pos = 0.0, 0

    for i, (c, p, t) in enumerate(zip(confs, predicts, targets)):
        rel = int(p == t)
        true_pos += rel

        res += true_pos / (i + 1) * rel

    res /= targets.shape[0] # FIXME: incorrect, not all test images depict landmarks
    return res

def train_step(train_loader, model, criterion, optimizer, epoch, lr_scheduler):
    print(f'epoch {epoch}')
    batch_time = AverageMeter()
    losses = AverageMeter()
    avg_score = AverageMeter()
    acc_score = AverageMeter()

    model.train()
    num_steps = len(train_loader)

    print(f'total batches: {num_steps}')

    end = time.time()
    lr = None

    for i, data in enumerate(train_loader):
        input_ = data['image']
        target = data['target']
        batch_size, _, _, _ = input_.shape

        output = model(input_.cuda())
        loss = criterion(output, target.cuda())
        confs, predicts = torch.max(output.detach(), dim=1)

        avg_score.update(GAP(predicts, confs, target), input_.size(0))
        losses.update(loss.data.item(), input_.size(0))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        lr = optimizer.param_groups[0]['lr']

        batch_time.update(time.time() - end)
        end = time.time()

        if i % LOG_FREQ == 0:
            print(f'{epoch} [{i}/{num_steps}]\t'
                  f'time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  f'loss {losses.val:.4f} ({losses.avg:.4f})\t'
                  f'GAP {avg_score.val:.4f} ({avg_score.avg:.4f})\t'
                  f'lr {lr:.8f}')

    print(f' * average GAP on train {avg_score.avg:.4f}')
    return avg_score.avg

### Generate Submission

In [ ]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)

def generate_submission(test_loader, model, label_encoder):
    sample_sub = pd.read_csv('../input/landmark-recognition-2020/sample_submission.csv')

    predicts_gpu, confs_gpu, _ = inference(test_loader, model)
    predicts, confs = predicts_gpu.cpu().numpy(), confs_gpu.cpu().numpy()
        

    labels = [label_encoder.inverse_transform(pred) for pred in predicts]
    print('labels', np.array(labels).shape)
    print('confs', np.array(confs).shape)

    sub = test_loader.dataset.df
    
    def concat(label: np.ndarray, conf: np.ndarray) -> str:
#         result = ' '.join([f'{L} {c}' for L, c in zip(label, conf)])
        # softmax conf
#         conf = softmax(conf)
#         print(label)
#         print(conf)
        result = ''
        for L, c in zip(label, conf):
            if L in private_counts:
                result = f'{L} {c}'
                break
        
        return result
    
    landmarks = [concat(label, conf) for label, conf in zip(labels, confs)]
    for i in range(len(landmarks)):
        if io_labels[i] == 0:
            landmarks[i] = ''
    
    sub['landmarks'] = landmarks
#     sub['landmarks'] = [concat(label, conf) if label[0] in private_counts else '' for label, conf in zip(labels, confs)]

    sample_sub = sample_sub.set_index('id')
    sub = sub.set_index('id')
    sample_sub.update(sub)

    sample_sub.to_csv('submission.csv')

### Process

In [ ]:
IN_KERNEL = os.environ.get('KAGGLE_WORKING_DIR') is not None
MIN_SAMPLES_PER_CLASS = 1 
BATCH_SIZE = 64
NUM_WORKERS = multiprocessing.cpu_count()
NUM_EPOCHS = 2
LOG_FREQ = 10
NUM_TOP_PREDICTS = 5
NUM_PUBLIC_TRAIN_IMAGES = 1580470
train_dir = '../input/landmark-recognition-2020/train/'
test_dir = '../input/landmark-recognition-2020/test/'

if __name__ == '__main__':
    # when re-running code, classes_num(num of landmark_ids) of the private train set  is less than the public train set, so we need to reload the original train.csv
    public_train = pd.read_csv('../input/glr-train-csv/train.csv') # The public train.csv
    train = pd.read_csv('../input/landmark-recognition-2020/train.csv')  # Treat as the private train.csv when submit to re-run code
    test = pd.read_csv('../input/landmark-recognition-2020/sample_submission.csv')
    
    global_start_time = time.time()
    private_counts = train.landmark_id.value_counts()
    print('Private train set landmark_ids lenght:', len(private_counts))
#     train_loader, test_loader, label_encoder, num_classes, train_len = load_data(train, test, train_dir, test_dir)
    train_loader, test_loader, label_encoder, num_classes, train_len = load_data(public_train, train, test, train_dir, test_dir)
#     print(label_encoder.inverse_transform([100]))

    model = EfficientNetEncoderHead(depth=0, num_classes=num_classes)
    try:
        model_path = '../input/efnb0-10-gap05917pth/efn-b0_10_GAP0.5917.pth'
        model.load_state_dict(torch.load(model_path, map_location='cpu'))
        print('Model found in {}'.format(model_path))
    except:
        print('Random initialize model!')
        
    model.cuda()
    
    is_finetune = False
    if train_len != 1580470:
        is_finetune = True
    # finetune
    if is_finetune:
        stop_weight_updata(model)
        criterion = nn.CrossEntropyLoss()

#         optimizer = optim.Adam(model.parameters(), lr=1e-6, weight_decay=1e-4)
        optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-6, weight_decay=1e-4)
        
        scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=len(train_loader)*NUM_EPOCHS, eta_min=1e-6)

        for epoch in range(1, 2):
            print('-' * 50)
            train_step(train_loader, model, criterion, optimizer, epoch, scheduler)
        

    print('inference mode')
    generate_submission(test_loader, model, label_encoder)

*Note: Will be publishing better kernels soon with more advanced techniques for landmark recognition.*
### More To Come. Stay Tuned. !!